# SQL on Apache Arrow

## References

- https://duckdb.org/docs/stable/guides/python/sql_on_arrow
- https://duckdb.org/docs/stable/guides/python/import_arrow
- https://duckdb.org/docs/stable/guides/python/export_arrow

In [8]:
import duckdb
import pyarrow as pa

# connect to an in-memory database
con = duckdb.connect()

my_arrow_table = pa.Table.from_pydict(
    {"i": [1, 2, 3, 4], "j": ["one", "two", "three", "four"]}
)

# query the Apache Arrow Table "my_arrow_table" and return as an Arrow Table
# results = con.execute("SELECT * FROM my_arrow_table WHERE i = 2").arrow()

ret = con.sql("SELECT * FROM my_arrow_table WHERE i = 2")

ret.show()

ret.arrow()

┌───────┬─────────┐
│   i   │    j    │
│ int64 │ varchar │
├───────┼─────────┤
│     2 │ two     │
└───────┴─────────┘



pyarrow.Table
i: int64
j: string
----
i: [[2]]
j: [["two"]]

In [ ]:
import duckdb
import pyarrow as pa
import tempfile
import pathlib
import pyarrow.parquet as pq
import pyarrow.dataset as ds

# connect to an in-memory database
con = duckdb.connect()

my_arrow_table = pa.Table.from_pydict(
    {"i": [1, 2, 3, 4], "j": ["one", "two", "three", "four"]}
)

# create example Parquet files and save in a folder
base_path = pathlib.Path(tempfile.gettempdir())
(base_path / "parquet_folder").mkdir(exist_ok=True)


pq.write_to_dataset(my_arrow_table, str(base_path / "parquet_folder"))

# link to Parquet files using an Arrow Dataset
my_arrow_dataset = ds.dataset(str(base_path / "parquet_folder/"))

# query the Apache Arrow Dataset "my_arrow_dataset" and return as an Arrow Table
# results = con.execute("SELECT * FROM my_arrow_dataset WHERE i = 2").arrow()
ret = con.sql("SELECT * FROM my_arrow_dataset WHERE i = 2")

ret.show()

ret.arrow()

┌───────┬─────────┐
│   i   │    j    │
│ int64 │ varchar │
├───────┼─────────┤
│     2 │ two     │
│     2 │ two     │
└───────┴─────────┘



pyarrow.Table
i: int64
j: string
----
i: [[2,2]]
j: [["two","two"]]

In [11]:
import duckdb
import pyarrow as pa
import tempfile
import pathlib
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pyarrow.compute as pc

# connect to an in-memory database
con = duckdb.connect()

my_arrow_table = pa.Table.from_pydict(
    {"i": [1, 2, 3, 4], "j": ["one", "two", "three", "four"]}
)

# create example Parquet files and save in a folder
base_path = pathlib.Path(tempfile.gettempdir())
(base_path / "parquet_folder").mkdir(exist_ok=True)
pq.write_to_dataset(my_arrow_table, str(base_path / "parquet_folder"))

# link to Parquet files using an Arrow Dataset
my_arrow_dataset = ds.dataset(str(base_path / "parquet_folder/"))

# define the filter to be applied while scanning
# equivalent to "WHERE i = 2"
scanner_filter = pc.field("i") == pc.scalar(2)

arrow_scanner = ds.Scanner.from_dataset(my_arrow_dataset, filter=scanner_filter)

# query the Apache Arrow scanner "arrow_scanner" and return as an Arrow Table
# results = con.execute("SELECT * FROM arrow_scanner").arrow()
ret = con.sql("SELECT * FROM arrow_scanner")

ret.show()

ret.arrow()

┌───────┬─────────┐
│   i   │    j    │
│ int64 │ varchar │
├───────┼─────────┤
│     2 │ two     │
│     2 │ two     │
│     2 │ two     │
└───────┴─────────┘



pyarrow.Table
i: int64
j: string
----
i: [[2,2,2]]
j: [["two","two","two"]]

In [ ]:
import duckdb
import pyarrow as pa

# connect to an in-memory database
con = duckdb.connect()

my_recordbatch = pa.RecordBatch.from_pydict(
    {"i": [1, 2, 3, 4], "j": ["one", "two", "three", "four"]}
)

my_recordbatchreader = pa.ipc.RecordBatchReader.from_batches(
    my_recordbatch.schema, [my_recordbatch]
)

# query the Apache Arrow RecordBatchReader "my_recordbatchreader" and return as an Arrow Table
# results = con.execute("SELECT * FROM my_recordbatchreader WHERE i = 2").arrow()
ret = con.sql("SELECT * FROM my_recordbatchreader WHERE i = 2")

ret.show()

ret.arrow()

┌───────┬─────────┐
│   i   │    j    │
│ int64 │ varchar │
├───────┼─────────┤
│     2 │ two     │
└───────┴─────────┘



pyarrow.Table
i: int64
j: string
----
i: []
j: []

In [24]:
import duckdb
import pyarrow as pa

# connect to an in-memory database
my_arrow = pa.Table.from_pydict({"a": [42]})


con = duckdb.connect(":memory:")

# create the table "my_table" from the DataFrame "my_arrow"
con.sql("CREATE TABLE IF NOT EXISTS my_table AS SELECT * FROM my_arrow")

# insert into the table "my_table" from the DataFrame "my_arrow"
con.sql("INSERT INTO my_table SELECT * FROM my_arrow")


ret = con.sql("SELECT * FROM my_table")
ret.df()

,a
0,42
1,42


In [26]:
import duckdb
import pyarrow as pa

my_arrow_table = pa.Table.from_pydict(
    {"i": [1, 2, 3, 4], "j": ["one", "two", "three", "four"]}
)

# query the Apache Arrow Table "my_arrow_table" and return as an Arrow Table
# results = duckdb.sql("SELECT * FROM my_arrow_table").arrow()
ret = duckdb.sql("SELECT * FROM my_arrow_table")

ret.show()

ret.arrow()

┌───────┬─────────┐
│   i   │    j    │
│ int64 │ varchar │
├───────┼─────────┤
│     1 │ one     │
│     2 │ two     │
│     3 │ three   │
│     4 │ four    │
└───────┴─────────┘



pyarrow.Table
i: int64
j: string
----
i: [[1,2,3,4]]
j: [["one","two","three","four"]]

In [ ]:
import duckdb
import pyarrow as pa

my_arrow_table = pa.Table.from_pydict(
    {"i": [1, 2, 3, 4], "j": ["one", "two", "three", "four"]}
)

# query the Apache Arrow Table "my_arrow_table" and return as an Arrow RecordBatchReader
chunk_size = 1_000_000
results = duckdb.sql("SELECT * FROM my_arrow_table").fetch_record_batch(chunk_size)


# Loop through the results. A StopIteration exception is thrown when the RecordBatchReader is empty
while True:
    try:
        # Process a single chunk here (just printing as an example)
        print(results.read_next_batch().to_pandas())
    except StopIteration:
        print("Already fetched all batches")
        break

In [31]:
import duckdb

# connect to an in-memory database
con = duckdb.connect()

con.execute("CREATE TABLE integers (i integer)")
con.execute(
    "INSERT INTO integers VALUES (0), (1), (2), (3), (4), (5), (6), (7), (8), (9), (NULL)"
)

# Create a relation from the table and export the entire relation as Arrow
rel = con.table("integers")
relation_as_arrow = rel.arrow()  # or .to_arrow_table()

# Or, calculate a result using that relation and export that result to Arrow
res = rel.aggregate("sum(i)").execute()

res.show()

res.arrow()  # or fetch_arrow_table()

┌────────┐
│ sum(i) │
│ int128 │
├────────┤
│     45 │
└────────┘



pyarrow.Table
sum(i): decimal128(38, 0)
----
sum(i): [[45]]